In [1]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

In [5]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune", skip_special_tokens=True)
)

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

In [64]:
tokenized_code = """
def find_max(values):\n    return max(values)
"""
pipeline([tokenized_code])[0]['summary_text']

Your max_length is set to 20, but you input_length is only 14. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


'Find the maximum value in values'

In [14]:
f = open('test-file.py', 'r')
file = f.read()
f.close()

In [15]:
file

"import newspaper\n\n\ndef find_max(values):\n    return max(values)\n\n\ndef average(values: list) -> int:\n\n    total = 0\n    for i in values:\n        total += i\n\n    return total/len(values)\n\n\ndef chunks(l, n):\n    d, r = divmod(len(l), n)\n    for i in range(n):\n        si = (d + 1) * (i if i < r else r) + d * (0 if i < r else i - r)\n        yield l[si:si + (d + 1 if i < r else d)]\n\n\ndef pull_articles_from_source(url, source, article_data=[]):\n    paper = newspaper.build(url)\n    i = 0\n    failed = 0\n    print(len(paper.articles))\n    paper.download_articles()\n    paper.parse_articles()  # remove articles that are too small (probably not articles)\n    print(len(paper.articles))\n    for article in paper.articles:\n        i += 1\n        if i > 10:\n            break\n        try:\n            # fail if the article is empty or less than 40 words\n            if article.text.isspace() or article.text == '' or len(article.text.split(' ')) < 40:\n                f

In [57]:
def pull_out_top_function(file):
    # find first def
    index = file.find('def ') # won't work if something ends in def like a variable
    if index == -1:
        return None

    # then grab all of the following text until there is no space between a newline character and the next character (signifiying the first code outside the function)
    sub_file = file[index:]
    for i in range(len(sub_file)-1):
        # you don't look for return statements because there can be more than one
        if sub_file[i] == '\n' and  not sub_file[i+1].isspace():
            return sub_file[:i]
        
    # def was found but 
    return sub_file.strip()

    
    

In [58]:
pull_out_top_function(file)

'def find_max(values):\n    return max(values)\n\n'

In [62]:
sub_file = file
new_file = file

while 'def ' in sub_file:

    function = pull_out_top_function(sub_file)

    original_function = function
    summary = 'this is the summary'

    # TODO won't work for functions within classes or things that need more spaces
    # need to find a way to dynamically get the correct spacing
    function = function.replace(':\n', f':\n    """\n    {summary}\n    """\n', 1)  # TODO this won't work for the case where theres a lot of type hints in the function definition and theres a ':\n'
    
    new_file = new_file.replace(original_function, function)
    
    sub_file = file[file.find(original_function) + len(original_function):]  # grab the rest of the file after the function


In [63]:
print(new_file)

import newspaper


def find_max(values):
    """
    this is the summary
    """
    return max(values)


def average(values: list) -> int:
    """
    this is the summary
    """

    total = 0
    for i in values:
        total += i

    return total/len(values)


def chunks(l, n):
    """
    this is the summary
    """
    d, r = divmod(len(l), n)
    for i in range(n):
        si = (d + 1) * (i if i < r else r) + d * (0 if i < r else i - r)
        yield l[si:si + (d + 1 if i < r else d)]


def pull_articles_from_source(url, source, article_data=[]):
    """
    this is the summary
    """
    paper = newspaper.build(url)
    i = 0
    failed = 0
    print(len(paper.articles))
    paper.download_articles()
    paper.parse_articles()  # remove articles that are too small (probably not articles)
    print(len(paper.articles))
    for article in paper.articles:
        i += 1
        if i > 10:
            break
        try:
            # fail if the article is empty or less than 40 